# MIT iQuHack Microsoft Challenge: Optimizing Quantum Oracles, Task 1

To work on this task,
1. Use the notebook for this task. Each of the notebooks in the repository has the code of the corresponding task.
2. Update your team name and Slack ID variables in the next code cell (you can use different Slack IDs for different tasks if different team members work on them, but make sure to use the same team name throughout the Hackathon). Do not change the task variable!
3. Work on your task in the cell that contains operation `Task1`! Your goal is to rewrite the code so that it maintains its correctness, but requires as few resources as possible. See `evaluate_results` function for details on how your absolute score for the task is calculated.
4. Submit your task using qBraid. Use the Share Notebook feature on qBraid (See File > Share Notebook) and enter the email rickyyoung@qbraid.com.   Once you click submit, if the share notebook feature works correctly, it should show that you receive no errors and the email you entered will disappear. 

Log in to Azure (once per session, don't need to do it if running from Azure workspace)

In [1]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E9GW54KA3 to authenticate.
The following tenants don't contain accessible subscriptions. Use 'az login --allow-no-subscriptions' to have tenant level access.
f107d4fe-a225-4cd3-a190-17b0d4f794c3 'Default Directory'
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "93e46887-8cc6-43a4-a4e0-e2209181e7d0",
    "id": "6d28879c-7c46-4b13-98e8-7da92da7e95e",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "d0ecd01b-d782-448e-bae0-c3cad0e0543a"
      },
      {
        "tenantId": "f702a9dc-ae48-4dc7-8f0a-8155a6dfa4e5"
      },
      {
        "tenantId": "94c4857e-1130-4ab8-8eac-069b40c9db20"
      }
    ],
    "name": "MIT iQuHack backup_JEEVESH KRISHNA_ARIGALA",
    "state": "Enabled",
    "tenantId": "93e46887-8cc6-43a4-a4e0-e2209181e7d0",
    "user": {
      "name": "a.jeeveshkrishna@gmail.com",
      "type": "user"
    }
  }
]


## Step 1. Write the code

In [3]:
# Run this code cell to import the modules required to work with Q# and Azure
import qsharp
from qsharp import azure

In [5]:
teamname="kodex_38"  # Update this field with your team name
task=["task1"]
slack_id="Jeevesh Krishna A"         # Update this field with Slack ID of the person who worked on this task as the troubleshooting contact

In [ ]:
# You don't need to run this cell, it defines Q# operations as Python types to keep IntelliSense happy
Task1_DumpMachineWrapper : qsharp.QSharpCallable = None
Task1_ResourceEstimationWrapper : qsharp.QSharpCallable = None

**The complete code for Task 1 should be in this cell.**   
This cell can include additional open statements and helper operations and functions if your solution needs them.  
If you define helper operations in other cells, they will not be picked up by the grader!

In [6]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;

// Task 1. Warm up with simple bit manipulation
// (input will contain 3 qubits)
operation Task1(input : Qubit[], target : Qubit) : Unit is Adj {
    let N = Length(input);
    use aux = Qubit[N - 1];
    within {
        for i in 0 .. N - 2 {
            CNOT(input[i], aux[i]);
            CNOT(input[i + 1], aux[i]);
        }
    } apply {
        Controlled X(aux, target);
    }
}

In [30]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;

// Task 1. Optimized with non-Clifford gates
// (input will contain 3 qubits)
operation Task1(input : Qubit[], target : Qubit) : Unit is Adj {
    CCNOT(input[0], input[1], target);
}


In [18]:
%%qsharp
// Wrapper operation that allows you to observe the effects of the marking oracle by running it on a simulator.
operation Task1_DumpMachineWrapper() : Unit {
    let N = 3;
    use (input, target) = (Qubit[N], Qubit());
    // Prepare an equal superposition of all input states in the input register.
    ApplyToEach(H, input);
    // Apply the oracle.
    Task1(input, target);
    // Print the state of the system after the oracle application.
    DumpMachine();
    ResetAll(input + [target]);
}

// Wrapper operation that allows to run resource estimation for the task.
// This operation only allocates the qubits and applies the oracle once, not using any additional gates or measurements.
operation Task1_ResourceEstimationWrapper() : Unit {
    let N = 3;
    use (input, target) = (Qubit[N], Qubit());
    Task1(input, target);
}

## Step 2. Run the code on a simulator to see what it does
You can also write your own code to explore the effects of the oracle (for example, applying it to different basis states and measuring the results).

In [19]:
# Note that in the output of this cell the target qubit corresponds to the rightmost bit
qsharp.config["dump.basisStateLabelingConvention"]="Bitstring"
qsharp.config["dump.phaseDisplayStyle"]="None"
# Uncomment the following line if you want to see only the entries with non-zero amplitudes
# qsharp.config["dump.truncateSmallAmplitudes"]=True
Task1_DumpMachineWrapper.simulate()

|0000⟩	0.35355339059327384 + 0𝑖
|0001⟩	0 + 0𝑖
|0010⟩	0.35355339059327384 + 0𝑖
|0011⟩	0 + 0𝑖
|0100⟩	0.35355339059327384 + 0𝑖
|0101⟩	0 + 0𝑖
|0110⟩	0.35355339059327384 + 0𝑖
|0111⟩	0 + 0𝑖
|1000⟩	0.35355339059327384 + 0𝑖
|1001⟩	0 + 0𝑖
|1010⟩	0.35355339059327384 + 0𝑖
|1011⟩	0 + 0𝑖
|1100⟩	0 + 0𝑖
|1101⟩	0.35355339059327384 + 0𝑖
|1110⟩	0 + 0𝑖
|1111⟩	0.35355339059327384 + 0𝑖

()

## Step 3. Evaluate the code using resource estimation

In [9]:
# If you're using this notebook in Azure Quantum hosted notebooks, remove the credential="CLI" parameter!
# If you're using this notebook in qBraid, keep it
qsharp.azure.connect(
    resourceId="/subscriptions/6d28879c-7c46-4b13-98e8-7da92da7e95e/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/iQuHackTest",
    location="eastus",
    credential="CLI")

Connecting to Azure Quantum...

Connected to Azure Quantum workspace iQuHackTest in location eastus.


[{'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 263325},
 {'id': 'ionq.qpu.aria-1', 'current_availability': {}, 'average_queue_time': 330610},
 {'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 3},
 {'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s1', 'current_availability': {}, 'average_queue_time': 257411},
 {'id': 'quantinuum.hqs-lt-s1-apival', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s2-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s1-sim', 'current_availability': {}, 'average_queue_time': 4413},
 {'id': 'quantinuum.hqs-lt-s2-sim', 'current_availability': {}, 'average_queue_time': 142},
 {'id': 'quantinuum.hqs-lt', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.qpu.h1-1', 'current

In [20]:
qsharp.azure.target("microsoft.estimator")

Loading package Microsoft.Quantum.Providers.Core and dependencies...
Active target is now microsoft.estimator


{'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0}

In [21]:
# Update job name to a more descriptive string to make it easier to find it in Job Management tab later
result = qsharp.azure.execute(Task1_ResourceEstimationWrapper, jobName="RE for the task 1")

Submitting Task1_ResourceEstimationWrapper to target microsoft.estimator...
Job successfully submitted.
   Job name: RE for the task 1
   Job ID: 32c7aa40-07cc-482b-ad94-922c72898e57
Waiting up to 30 seconds for Azure Quantum job to complete...
[6:16:02 PM] Current job status: Executing
[6:16:08 PM] Current job status: Succeeded


In [22]:
# If you need to pull up the results of an old job, use its job ID with qsharp.azure.output command
# result = qsharp.azure.output("...")
result

Physical qubits,18110,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 2430 physical qubits to implement the algorithm logic, and 15680 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Runtime,46us 800ns,"Total runtime This is a runtime estimate (in nanosecond precision) for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (3us 600ns) multiplied by the 3 logical cycles to run the algorithm. If however the duration of a single T factory (here: 36us 400ns) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
Logical algorithmic qubits,15,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the \(Q_{\rm alg} = 4\) logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 15$ logical qubits."
Algorithmic depth,3,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 0 single-qubit measurements, the 0 arbitrary single-qubit rotations, and the 0 T gates, three multi-qubit measurements for each of the 1 CCZ and 0 CCiX gates in the input program, as well as No rotations in algorithm multi-qubit measurements for each of the 0 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,13,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 3. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Number of T states,4,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 0 T gates, four T states for each of the 1 CCZ and 0 CCiX gates, as well as No rotations in algorithm for each of the 0 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,4,Number of T factories capable of producing the demanded 4 T states during the algorithm's runtime The total number of T factories 4 that are executed in parallel is computed as \(\left\lceil\dfrac{4\;\text{T states} \cdot 36us 400ns\;\text{T factory duration}}{1\;\text{T states per T factory} \cdot 46us 800ns\;\text{algorithm runtime}}\right\rceil\)
Number of T factory invocations,1,"Number of times all T factories are invoked In order to prepare the 4 T states, the 4 copies of the T factory are repeatedly invoked 1 times."
Physical algorithmic qubits,2430,Number of physical qubits for the algorithm after layout The 2430 are the product of the 15 logical qubits after layout and the 162 physical qubits that encode a single logical qubit.
Physical T factory qubits,15680,"Number of physical qubits for the T factories Each T factory requires 3920 physical qubits and we run 4 in parallel, therefore we need $15680 = 3920 \cdot 4$ qubits."
Required logical qubit error rate,2.56e-6,The minimum logical qubit error rate required to run the algorithm within the error budget The minimum logical qubit error rate is obtained by dividing the logical error probability 5.00e-4 by the product of 15 logical qubits and the total cycle count 13.


In [23]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth


In [24]:
evaluate_results(result)

Logical algorithmic qubits = 15
Algorithmic depth = 3
Score = 45


45